In [4]:
import re
import random
import pandas as pd
from docx import Document
from tkinter import Tk, messagebox
from tkinter.filedialog import askopenfilename, asksaveasfilename
import genanki

def read_and_parse_questions(docx_path):
    try:
        document = Document(docx_path)
        text = '\n'.join([para.text for para in document.paragraphs])
    except Exception as e:
        print(f"Error reading document: {e}")
        return None
    
    question_blocks = re.findall(r'\d+\..+?(?=\d+\.|\Z)', text, re.DOTALL)
    parsed_questions = []
    
    for block in question_blocks:
        match = re.match(r'(\d+)\..+?([ABCD]\).+?)(?=[ABCD]\)|Answer Key|\Z)', block, re.DOTALL)
        if match:
            question_number = match.group(1)
            question_text = match.group(2)
            choices = re.findall(r'([ABCD]\).+?)(?=[ABCD]\)|Answer Key|\Z)', question_text, re.DOTALL)
            correct_answer = re.search(r'Answer Key\s*(\w)', block)
            if correct_answer:
                correct_answer = correct_answer.group(1)
            else:
                print("Answer key not found.")
                return None
            parsed_questions.append({
                'Question Number': question_number,
                'Question': choices[0][3:].strip(),
                'A': choices[1][3:].strip(),
                'B': choices[2][3:].strip(),
                'C': choices[3][3:].strip(),
                'D': choices[4][3:].strip(),
                'Correct Answer': correct_answer
            })
        else:
            print(f"Format issue detected in question block: {block}")
            return None
    return parsed_questions


def save_to_excel(questions, excel_path):
    try:
        df = pd.DataFrame(questions)
        df.to_excel(excel_path, index=False)
        print(f"Questions saved to {excel_path}")
    except Exception as e:
        print(f"Error saving to Excel: {e}")
        return False
    return True

def preview_excel(excel_path):
    try:
        df = pd.read_excel(excel_path)
        print(df.head())
        if input("Is the Excel file correct? (y/n): ").lower() != 'y':
            print("Please correct the Excel file and run the script again.")
            exit()
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        exit()

def create_anki_deck_from_excel(excel_path, deck_name, output_path):
    try:
        df = pd.read_excel(excel_path)
        questions = df.to_dict('records')
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return

    deck = genanki.Deck(random.randrange(1 << 30, 1 << 31), deck_name)
    model = genanki.Model(
        random.randrange(1 << 30, 1 << 31),
        'Simple Model',
        fields=[
            {'name': 'Question'},
            {'name': 'A'},
            {'name': 'B'},
            {'name': 'C'},
            {'name': 'D'},
            {'name': 'Correct Answer'},
        ],
        templates=[{
            'name': 'Card 1',
            'qfmt': '{{Question}}<br><br>A) {{A}}<br>B) {{B}}<br>C) {{C}}<br>D) {{D}}',
            'afmt': '{{Question}}<br><br>A) {{A}}<br>B) {{B}}<br>C) {{C}}<br>D) {{D}}<br><br><b>Correct Answer: {{Correct Answer}}</b>',
        }]
    )

    for q in questions:
        note = genanki.Note(
            model=model,
            fields=[q['Question'], q['A'], q['B'], q['C'], q['D'], q['Correct Answer']]
        )
        deck.add_note(note)
    
    try:
        genanki.Package(deck).write_to_file(output_path)
        print(f"Anki deck saved to {output_path}")
    except Exception as e:
        print(f"Error saving Anki deck: {e}")

def main():
    Tk().withdraw()
    docx_path = askopenfilename(filetypes=[("Word Documents", "*.docx")])
    if not docx_path:
        print("No file selected.")
        return

    questions = read_and_parse_questions(docx_path)
    if not questions:
        print("Failed to parse questions.")
        return
    
    excel_path = asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel Files", "*.xlsx")])
    if not excel_path:
        print("No save location selected.")
        return

    if not save_to_excel(questions, excel_path):
        return
    
    preview_excel(excel_path)

    anki_path = asksaveasfilename(defaultextension=".apkg", filetypes=[("Anki Package", "*.apkg")])
    if not anki_path:
        print("No save location selected.")
        return

    deck_name = input("Enter the name of the Anki deck: ")
    create_anki_deck_from_excel(excel_path, deck_name, anki_path)

if __name__ == "__main__":
    main()


Failed to parse questions.
